<a href="https://colab.research.google.com/github/kiat/Cloud-Computing/blob/main/Colab_SocialMedia_Example_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,

In [ ]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

sc = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
# Download the data
! wget -q https://www.cs.utexas.edu/~kiat/datasets/social-media/person_knows_person.csv
! wget -q https://www.cs.utexas.edu/~kiat/datasets/social-media/person_likes_post.csv
! wget -q https://www.cs.utexas.edu/~kiat/datasets/social-media/post_hasCreator_person.csv
! wget -q https://www.cs.utexas.edu/~kiat/datasets/social-media/comment_hasCreator_person.csv
! wget -q https://www.cs.utexas.edu/~kiat/datasets/social-media/comment_replyOf_post.csv

In [ ]:
! ls -la

total 33568
drwxr-xr-x 1 root root     4096 Oct  4 16:38 .
drwxr-xr-x 1 root root     4096 Oct  4 16:37 ..
-rw-r--r-- 1 root root  7398677 Jun 19  2024 comment_hasCreator_person.csv
-rw-r--r-- 1 root root  3392677 Jun 19  2024 comment_replyOf_post.csv
drwxr-xr-x 4 root root     4096 Oct  2 13:36 .config
-rw-r--r-- 1 root root   232246 Jun 19  2024 person_knows_person.csv
-rw-r--r-- 1 root root 21372777 Jun 19  2024 person_likes_post.csv
-rw-r--r-- 1 root root  1950946 Jun 19  2024 post_hasCreator_person.csv
drwxr-xr-x 1 root root     4096 Oct  2 13:36 sample_data


In [ ]:
!pwd

/content


In [ ]:
# Set your file path here
path="file:///content/"

# You have 5 files

# comment_hasCreator_person.csv
# comment_replyOf_post.csv
# person_knows_person.csv
# person_likes_post.csv
# post_hasCreator_person.csv

fileCommentHasCreator= path+"comment_hasCreator_person.csv"
fileComment_replyOf_post = path+"comment_replyOf_post.csv"
filePerson_knows_person= path+"person_knows_person.csv"
filePerson_likes_post = path + "person_likes_post.csv"
filePost_hasCreator_person = path + "post_hasCreator_person.csv"

#Create RDDs

In [ ]:
def getRDD(mfile, sc):
		lines = sc.textFile(mfile)
		# Removing the Header Line
		linesHeader = lines.first()
		header = sc.parallelize([linesHeader])
		linesWithOutHeader = lines.subtract(header)
		myRDD = linesWithOutHeader.map(lambda x: x.split('|'))
		return myRDD

In [ ]:
	# Create RDDs from files
	commentHasCreator = getRDD(fileCommentHasCreator, sc)
	comment_replyOf_post = getRDD(fileComment_replyOf_post, sc)
	person_knows_person = getRDD(filePerson_knows_person, sc)
	person_likes_post = getRDD(filePerson_likes_post, sc)
	post_hasCreator_person = getRDD(filePost_hasCreator_person, sc)

In [ ]:
commentHasCreator.take(1)

[['20', '913']]

# Create Dataframes

In [ ]:
commentHasCreatorDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(fileCommentHasCreator)
comment_replyOf_postDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(fileComment_replyOf_post)
person_knows_personDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePerson_knows_person)
person_likes_postDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePerson_likes_post)
post_hasCreator_personDD = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(filePost_hasCreator_person)


In [ ]:
commentHasCreatorDD.show()
comment_replyOf_postDD.show()
person_knows_personDD.show()
person_likes_postDD.show()
post_hasCreator_personDD.show()

+----------+---------+
|Comment.id|Person.id|
+----------+---------+
|         0|       74|
|        10|      832|
|        20|      913|
|        30|      457|
|        40|      956|
|        50|       41|
|        60|      453|
|        70|      832|
|        80|        6|
|        90|        6|
|       100|      103|
|       110|      547|
|       120|      962|
|       130|       99|
|       140|      452|
|       150|       99|
|       160|        6|
|       170|       48|
|       180|      941|
|       190|       40|
+----------+---------+
only showing top 20 rows

+----------+-------+
|Comment.id|Post.id|
+----------+-------+
|         0|      0|
|        10|      0|
|        30|      0|
|        70|      0|
|       100|     10|
|       110|     10|
|       140|     10|
|       150|     10|
|       180|     10|
|       240|     20|
|       250|     20|
|       330|     20|
|       340|     30|
|       350|     30|
|       420|     30|
|       470|     40|
|       480|     40|
| 

# Task 1: Consider that total user activities count to be the summation of the number of user's posts, comments, likes and having friends. Which top-10 users have the highest activity counts?



# Task 2: Which users wrote the top-10 highest active posts? Top-10 posts are posts that have the highest number of comments and likes.



# Task 3: Find the top-10 good friends. Two users are considered to be good friends on this social media when they do a lots of likes and comments on each others posts.

We would like to find the top-10 of these good friend pairs. Find and print the top-10 good friends based on total number of likes and comments. For each pair of friends you can count up a total score number (sum of all likes and comments), then going over each friendship pair combination, we want to have the top-10.



# Task 4: Consider only the users who have at least 20 friends. Who are the two users that have the most number of common friends?
Implement it in RDD and Spark Dataframe